# ライントレース+物体検知

## 物体検知側の下準備

In [8]:
from jetbot import ObjectDetector

# 物体検知モデルの読み込み
model_od = ObjectDetector('../note_wavashare/object_following/ssd_mobilenet_v2_coco.engine')

In [9]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)  # カメラの起動

In [10]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

mean_obj = 255.0 * np.array([0.485, 0.456, 0.406])
stdev_obj = 255.0 * np.array([0.229, 0.224, 0.225])
normalize_obj = torchvision.transforms.Normalize(mean_obj, stdev_obj)

def preprocess_obj(camera_value):
    global device, normalize_obj
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

## ライントレース側の下準備

In [11]:
# インポート
import torchvision
import torch

# 自分で作成したライントレースのモデルを読み込む
model_rf = torchvision.models.resnet18(pretrained=False)
model_rf.fc = torch.nn.Linear(512, 2)
model_rf.load_state_dict(torch.load('../note_nvidia/road_following/best_steering_models/[nvidia_circuit-ver1.2]_res18.pth'))

<All keys matched successfully>

In [12]:
# GPU側に転送
device = torch.device('cuda')
model_rf = model_rf.to(device)
model_rf = model_rf.eval().half()

In [13]:
# カメラから読み込む映像への前処理を実装
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [14]:
# カメラからの映像をセルの下に出力
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

# camera = Camera()
# image_widget = ipywidgets.Image()
# traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# display(image_widget)

In [15]:
# 駆動系周りの準備
from jetbot import Robot

robot = Robot()  # 駆動系を制御できるモジュールのインポート

# 制御するバーの設定
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

# 描画
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

# 自Jetbotにおいては、以下の設定だと良い感じに動く
# speed gain = 0.1
# steering gain = 0.05
# steering kd = 0
# steering gain = 0

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.05, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [16]:
# 現在の動作を示すバーを描画
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

## 本処理

In [18]:
from jetbot import bgr8_to_jpeg
import time

blocked_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = ipywidgets.Image(format='jpeg', width=300, height=300)
label_widget = ipywidgets.IntText(value=1, description='tracked label')
speed_widget = ipywidgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = ipywidgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(ipywidgets.VBox([
    ipywidgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

angle = 0.0
angle_last = 0.0

def execute(change):
   
    global angle, angle_last
    image = change['new']  # 映像読み込み
    #----------------------------ライントレース部分--------------------------------------------------#
    xy = model_rf(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    #----------------------------------------------------------------------------------------------------#
    
    #--------------------------------------------物体検知部分------------------------------------------------#
    # カメラの映像内において、検知出来た全ての物体を、変数detectionに入れる
    detections = model_od(image)
    
    # 出力されているカメラ映像の上に、物体の枠(バウンディングボックス)を描画する
    label_id=[]
    for i, det in enumerate(detections[0]):
        # バウンディングボックスの描画
        bbox = det['bbox']
        image = cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
        
        # ラベルIDを表示
        # label_id = det['label'] と記述してしまうと、det['label']が空の場合、label_idは宣言されない扱いになってしまうので、Noneが代入された配列として宣言する
        if(det['label']==False):
            label_id.append("null")
            print("label_id is null")
        else:
            label_id.append(str(det['label']))
        
        # {この辺に、「ラベルIDと一致する名前を持ってくる」みたいな処理}
        cv2.putText(image, "id:"+str(det['label']), (int(width * bbox[0]), int(height * bbox[1]+3)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (36,255,12), 2)
        
    print(label_id)
    
    # select detections that match selected class label
      # 追跡する物体を設定する(ユーザがlabel番号を指定、その番号をmatching_detectionsに代入)
      # label_widget.value : ユーザが指定するやつ、1とか2とか設定できるやつ
      # d['label'] : 
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    # 検知した物体によって動きを変える
    
    pause_flag = str(44) in label_id  # bottleを検知したら一時停止
    if(pause_flag == True):
        print("pause")
        robot.stop()
        time.sleep(0.1)
        
    
    # 出力されているカメラ映像の更新
    image_widget.value = bgr8_to_jpeg(image)
    
    
execute({'new': camera.value})
camera.observe(execute, names='value')

['73', '31']
['73', '31']


['73']
['73']
['73', '31']
['73']
['73']
['73']
['73']
['73']
['73']
['73', '31']
['73', '72']
['73']
['73']
['73']
['73']
['73']
['73']
['73']
['73']
['73', '31']
['73']
['73']
['73']
['73', '31']
['73']
['73', '31']
['73']
['73', '31']
['73']
['73']
['73']
['73', '72']
['72']
['73']
[]
['74', '77', '73']
['77', '73']
['74']
['74', '72']
['1', '75', '75']
['1']
['76', '1', '76', '74']
['76', '1']
['1', '76', '1']
['76', '1']
['1', '76', '74']
['76', '33', '1']
['1', '76', '74', '74']
['1', '33']
['1', '76', '76', '74', '74']
['65', '65']
['1']
['1']
['62']
[]
['62']
['1']
['70', '18', '74']
['80']
['70', '62']
[]
['70']
[]
['70']
[]
['70']
[]
['1']
['1']
['62', '1', '1', '62']
['1']
['1', '44']
[]
['44', '73']
['73']
['44', '67']
['67', '44', '90', '90']
pause
['44', '67']
[]
['44', '67', '73']
['73']
['44', '67']
['67', '44']
pause
['44', '67']
['44', '67']
pause
['44', '67']
['67', '44']
pause
['44', '67']
['44', '67', '90']
pause
['44', '67']
['67', '44']
pause
['44', '67']
['86']


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [10]:
# camera.observe(execute, names='value')

In [19]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
camera.stop()
robot.stop()

['1']
['1']
['1', '86']
['1']


In [13]:
camera.stop()
robot.stop()